In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import h5py

# 2D-Dataset: braindr
- five slices from each case
- the time is messerued
- the citicens can choose fail or pass each image
- 3609 images with minimum of 19 rates!

In [11]:
# get csv  
path_to_braindr_ratings = '../../project/data/braindr_data.csv'
braindr_data_csv = pd.read_csv(path_to_braindr_ratings)
braindr_data_csv['image_name'] = [row.split('_')[0] + '_' + row.split('_')[1] for row in braindr_data_csv['image_id']]
braindr_data_csv['image_pos'] = [row.split('_')[-1] for row in braindr_data_csv['image_id']]
braindr_data_csv



In [4]:
# load data of braindr_slice_data.npz
raw_data = np.load('../data/braindr_slice_data.npz', allow_pickle=True)
data_image_ids = raw_data['labels']

['X', 'labels', 'X_vgg']

In [12]:
# maps braindr image ids with csv ids and sorts for mean votes
mean_votes = braindr_data_csv.groupby('image_id')['image_name','image_pos','vote'].mean()
mean_votes['image_name'] = braindr_data_csv.groupby('image_id')['image_name'].min()
mean_votes['image_pos'] = braindr_data_csv.groupby('image_id')['image_pos'].min()
mean_votes['sort_values'] = [list(data_image_ids).index(i) if i in list(data_image_ids) else None for i in mean_votes.index.values]
mean_votes = mean_votes[mean_votes['sort_values'].isnull() == False]
mean_votes_sorted_vote = mean_votes.sort_values('vote')

data_meanvotes = mean_votes
mean_votes_sorted_vote

vote            image_name image_pos  \
image_id                                                                 
sub-NDARJY141RFE_T1w__ax_73   0.047619  sub-NDARJY141RFE_T1w        73   
sub-NDARTT759PC5_T1w__ax_90   0.050000  sub-NDARTT759PC5_T1w        90   
sub-NDARXT792GY8_T1w__ax_186  0.050000  sub-NDARXT792GY8_T1w       186   
sub-NDARWT389BYY_T1w__ax_74   0.083333  sub-NDARWT389BYY_T1w        74   
sub-NDARHF904CWB_T1w__ax_99   0.090909  sub-NDARHF904CWB_T1w        99   
...                                ...                   ...       ...   
sub-NDARGB441VVD_T1w__ax_146  1.000000  sub-NDARGB441VVD_T1w       146   
sub-NDARGB441VVD_T1w__ax_66   1.000000  sub-NDARGB441VVD_T1w        66   
sub-NDARGB441VVD_T1w__ax_86   1.000000  sub-NDARGB441VVD_T1w        86   
sub-NDARFW130NGG_T1w__ax_78   1.000000  sub-NDARFW130NGG_T1w        78   
sub-NDARWJ019VGU_T1w__ax_145  1.000000  sub-NDARWJ019VGU_T1w       145   

                              sort_values  
image_id                                   
sub-NDARJY141RFE_T1w__ax_73        1438.0  
sub-NDARTT759PC5_T1w__ax_90        2584.0  
sub-NDARXT792GY8_T1w__ax_186       3221.0  
sub-NDARWT389BYY_T1w__ax_74        3021.0  
sub-NDARHF904CWB_T1w__ax_99        1114.0  
...                                   ...  
sub-NDARGB441VVD_T1w__ax_146        912.0  
sub-NDARGB441VVD_T1w__ax_66         913.0  
sub-NDARGB441VVD_T1w__ax_86         914.0  
sub-NDARFW130NGG_T1w__ax_78         863.0  
sub-NDARWJ019VGU_T1w__ax_145       2960.0  

[3607 rows x 4 columns]

In [13]:
#function for downloading images
import urllib
import matplotlib.pyplot as plt
def get_image(img, path):
    url = "https://s3-us-west-2.amazonaws.com/akeshavan-mindcontrol/braindr/{}.jpg".format(img)
    urllib.request.urlretrieve(url,path + img+'.jpg')


In [ ]:
# download all images 
# this will download serveral gbs of images
image_ids = mean_votes_sorted_vote['image_name'].index

download_path = '../data/image_download/'
for id in image_ids :
    get_image(id, download_path)

In [3]:
# data from 3d to 2d images
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

In [15]:
# put all images with labels from folder in  grayscale dimensions in numpy array
path = '../data/image_download/'
ftype = '.jpg'
import glob
from PIL import Image
filelist = glob.glob(path + '*' + ftype)
# for motion
data = np.array([(rgb2gray(np.array(Image.open(fname))),fname.replace(path, '').replace(ftype,"")) for fname in filelist])

# for gaussian
#data = np.array([(rgb2gray(np.array(Image.open(fname)))) for fname in filelist])


In [12]:
# only for motion, get ids from best 1200
best_1200 = mean_votes_sorted_vote['image_name']
best_1200 = mean_votes_sorted_vote.index[-1200:]
print(best_1200)

Index(['sub-NDARED047DTH_T1w__ax_93', 'sub-NDARTU777GVV_T1w__ax_50',
       'sub-NDARJM708VGE_T1w__ax_151', 'sub-NDARXX895FNJ_T1w__ax_165',
       'sub-NDARLR030EW6_T1w__ax_156', 'sub-NDARBE641DGZ_T1w__ax_117',
       'sub-NDARHP558MGA_T1w__ax_76', 'sub-NDARHP558MGA_T1w__ax_96',
       'sub-NDARHM615PJH_T1w__ax_85', 'sub-NDARJM239GN4_T1w__ax_99',
       ...
       'sub-NDARFY612EMR_T1w__ax_81', 'sub-NDARFZ296UNG_T1w__ax_63',
       'sub-NDARWV405ZW0_T1w__ax_145', 'sub-NDARGA967MGC_T1w__ax_81',
       'sub-NDARGB040MGR_T1w__ax_79', 'sub-NDARGB441VVD_T1w__ax_146',
       'sub-NDARGB441VVD_T1w__ax_66', 'sub-NDARGB441VVD_T1w__ax_86',
       'sub-NDARFW130NGG_T1w__ax_78', 'sub-NDARWJ019VGU_T1w__ax_145'],
      dtype='object', name='image_id', length=1200)


In [5]:
# get images with (320, 320)  shape
# for motion
images = []
for i in data:
    if i[0].shape  == (320,320):
        images.append(np.array(i))
        
images = np.array(images)

## for gaussian
# images = []
# for i in data:
#     if i.shape  == (320,320):
#         images.append(np.array(i))
        
# images = np.array(images)



In [11]:
print(images.shape)

(1894, 320, 320)


In [14]:
#for motion, gets from 320 shape images the ones in best 1200 votes
images_final = []
for i in best_1200:
    for row in images:
        if i == row[1]:
        
            images_final.append(row[0])
          
    
images_final = np.array(images_final)
print(images_final.shape)

(516, 320, 320)


In [13]:
# split in val and train

#for motion
X_val = train_test_split(images_final, test_size = 0.15, random_state = 1, shuffle=True)
#for gaussian
# X_train, X_val = train_test_split(images, test_size = 0.20, random_state = 1, shuffle=True)
# X_train, X_test = train_test_split(X_train, test_size = 0.25, random_state = 1, shuffle=True)

In [16]:
save motion
hf = h5py.File('motion_data.hdf5', 'w')
hf.create_dataset('train_data', data=X_train/255)
hf.create_dataset('val_data', data=X_val/255)
hf.close

# # save gaussian
# hf = h5py.File('gaussian_data.hdf5', 'w')
# hf.create_dataset('train_data', data=X_train/255)
# hf.create_dataset('val_data', data=X_val/255)
# hf.create_dataset('test_data', data=X_test/255)
# hf.close

<bound method File.close of <HDF5 file "gaussian_data.hdf5" (mode r+)>>